## TFLite inference

Test TensorFlow Lite inference on an input image.

In [1]:
import os
import cv2
import numpy as np
from os.path import join
import tensorflow as tf
import pickle
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import torch
import numpy as np
import torch.utils.data
from STEAL.models.casenet import casenet101 as CaseNet101
from STEAL.coarse_to_fine.input_reader import InputReaderSemMatDemo
from STEAL.contours import ContourBox

import cv2

import STEAL.utils.vis_utils as vs_utils
import os
from STEAL.utils.VisualizerBox import VisualizerBox
import matplotlib.pyplot as plt

In [3]:
# Load TFLite model and allocate tensors.

model1= r"mobilenetv2_unet_edge_quant.tflite"

interpreter = tf.lite.Interpreter(model_path=model1)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

interpreter.allocate_tensors()

[{'name': 'input_image', 'index': 0, 'shape': array([  1, 224, 224,   3]), 'shape_signature': array([ -1, 224, 224,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 183, 'shape': array([  1, 224, 224,   1]), 'shape_signature': array([ -1, 224, 224,   1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


### Test on an image

In [ ]:
#PATH = "Z://AIVie//Patent//siamese-contrastive-pyimage//experiment//Foreground//dataset//AHP//train//dataset//zoomed//images//COCO_val2014_000000532812_sp_00.png"
PATH = r"C:\Users\alext\Desktop\dataset\AHP\reserved\JPEGImages\COCO_train2014_000000015546_sp_00.jpg"
#PATH = "test_images//t7.jpg"
INPUT_SIZE = (224, 224)

test_img = cv2.imread(PATH)
shp = test_img.shape
test_img = test_img[:,:,::-1]
test_img = cv2.resize(test_img, INPUT_SIZE)


# Test model on the input image.
input_shape = input_details[0]['shape']
input_data = np.array((test_img.astype(np.float32) / 255.0)[np.newaxis, :, :, :])

In [ ]:
input_data.shape

In [ ]:
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data.shape)


### Profile execution

In [ ]:
import time

n = 100
start_time = time.time()
for _ in range(n):
    interpreter.invoke()

print("FPS:", round(n/(time.time() - start_time), 2))

### Visualize predictions

In [ ]:
import PIL
threshold = 100



out_img = np.squeeze(output_data)
out_img = cv2.resize(out_img, (shp[1],shp[0]))

print(out_img.shape)
print(cv2.cvtColor(out_img, cv2.COLOR_GRAY2BGR).shape)

test_img = cv2.imread(PATH)


plt.figure(figsize=(20,10))
plt.axis('off')
plt.imshow(cv2.resize(test_img, (shp[1],shp[0])))
plt.imshow(out_img*255 > threshold, alpha=0.3)
plt.savefig(r'C:\Users\alext\Desktop\dataset\AHP\result\AHPwithZoom_Ori_dicecoef_pic1.jpg')

plt.show()

In [ ]:
import cv2
import random


image1 = cv2.imread(r'C:\Users\alext\Desktop\dataset\AHP\result\AHPwithZoom_dicecoef_pic1.jpg')
image2 = cv2.imread(r'C:\Users\alext\Desktop\dataset\AHP\result\AHPwithZoom_16k_dicecoef_pic5.jpg')
image3 = cv2.imread(r'C:\Users\alext\Desktop\dataset\AHP\result\AHPwithZoom_Ori_dicecoef_pic1.jpg')
image4 = cv2.imread(r'C:\Users\alext\Desktop\dataset\AHP\result\AHPwithZoom_16k_dicecoef_pic7.jpg')


fig = plt.figure(figsize=(20, 20))

# f, axs = plt.subplots(1,2,figsize=(20,20))

# ax1 = plt.subplot(1, 2, 1), plt.imshow(image1, 'gray')
# ax2 = plt.subplot(1, 2, 2), plt.imshow(image2, 'gray')

# fig = plt.figure()
# ax1 = fig.add_subplot(221)
# ax2 = fig.add_subplot(222)
# ax1.title.set_text('First Plot')
# ax2.title.set_text('Second Plot')
# plt.imshow(image1, 'gray')
# plt.imshow(image2, 'gray')

#image 1
fig.add_subplot(2, 1, 1)

plt.imshow(image3)
plt.axis('off')
plt.title("Before expand",fontsize = 40)

#image 2
fig.add_subplot(2, 1, 2)
plt.imshow(image4)
plt.axis('off')
plt.title("After expand",fontsize = 40)

# #image 3
# fig.add_subplot(2,2,3)
# plt.imshow(image3)
# plt.axis('off')
# plt.title("Before expand",fontsize = 40)

# #image 4
# fig.add_subplot(2,2,4)
# plt.imshow(image4)
# plt.axis('off')
# plt.title("After expand",fontsize = 40)

fig.savefig('Expand_diff2.png', dpi=fig.dpi)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def process(filename: str=None) -> None:
    """
    View multiple images stored in files, stacking vertically

    Arguments:
        filename: str - path to filename containing image
    """
    image = mpimg.imread(filename)
    plt.figure()
    plt.imshow(image)

images = os.listdir(r'C:\Users\alext\Desktop\dataset\AHP\result')


for file in images:
    file = os.path.join(r'C:\Users\alext\Desktop\dataset\AHP\result',file)
    process(file)

plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("common X")
plt.ylabel("common Y")

## Test on Video

In [4]:
import time


In [4]:
Model_Path = r"C:\Users\alext\Jupyter\mobilenetv2_unet_edge_quant.tflite"
input_path = (r'C:\Users\alext\Desktop\dataset\HED2.mp4')

In [5]:
interpreter = tf.lite.Interpreter(model_path=Model_Path)
interpreter.allocate_tensors()


In [6]:
alpha = 0.5
beta = (1.0 - alpha)


In [7]:
cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = None

while (True):
    
    ok, frame = cap.read()

    if not ok:
        break

    ################
    #model start
    resize_img = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_CUBIC)

    input_shape = input_details[0]['shape']
    input_data = np.array((resize_img.astype(np.float32) / 255.0)[np.newaxis, :, :, :])

    #Test model
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
        
    #Masking
    masking = np.squeeze(output_data)
    masking = cv2.resize(masking, (frame.shape[1],frame.shape[0]))
    masking= cv2.cvtColor(masking, cv2.COLOR_GRAY2BGR)
    masking = masking * 255
            
    #Add masking to Video
    dst = cv2.addWeighted(frame,0.8,masking,0.6,0.0,dtype=cv2.CV_8U) 


    
    ##################### 
    #writer
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")

        writer = cv2.VideoWriter(r'C:\Users\alext\Desktop\output.mp4', fourcc, 30,(frame.shape[1],frame.shape[0]), True)

    if writer is not None:
        
        writer.write(dst)
    ##############################
    # visualisation
    cv2.namedWindow("cv_image", cv2.WINDOW_NORMAL)
    cv2.imshow("cv_image",dst)


    if cv2.waitKey(1) & 0xFF == ord('q') :
        break
        
    if cv2.waitKey(1) & 0xFF == ord('p'):
        cv2.waitKey(-1) #wait until any key is pressed

if writer is not None:
    writer.release()

    
# result.release()
cap.release()
cv2.destroyAllWindows()


In [ ]:
cap = cv2.VideoCapture(input_path)
ok, frame_image = cap.read()
original_image_height, original_image_width, _ = frame_image.shape
